In [3]:
# Experiment to compare PyTorch, and TensorFlow on CPU runtime
import numpy as np
import torch
import tensorflow as tf
import time

# Function to measure time for PyTorch (CPU)
def pytorch_cpu_test(matrix):
    # Ensure strict symmetry and use float64 for precision
    matrix = (matrix + matrix.T) / 2
    torch_matrix = torch.tensor(matrix, dtype=torch.float64)  # Use float64 for better stability

    try:
        start_time = time.time()
        eigenvalues, eigenvectors = torch.linalg.eigh(torch_matrix)  # Preferred method
        end_time = time.time()
    except RuntimeError:
        print("Fallback to symeig due to RuntimeError.")
        start_time = time.time()
        eigenvalues, eigenvectors = torch.symeig(torch_matrix, eigenvectors=True)  # Fallback
        end_time = time.time()

    return end_time - start_time, None, None  # Skip memory tracking for CPU

# Function to measure time for TensorFlow (CPU)
def tensorflow_cpu_test(matrix):
    tf_matrix = tf.convert_to_tensor(matrix, dtype=tf.float32)  # Ensure TensorFlow uses CPU
    start_time = time.time()
    eigenvalues, eigenvectors = tf.linalg.eigh(tf_matrix)
    end_time = time.time()
    return end_time - start_time, None, None  # TensorFlow memory tracking disabled for CPU

# Testing matrix sizes
matrix_sizes = [10, 100, 500, 1000, 2500, 5000]
results = {"size": [], "pytorch": [], "tensorflow": []}

# Run experiments
for size in matrix_sizes:
    print(f"Testing matrix size: {size}x{size}")
    matrix = np.random.rand(size, size)
    matrix = (matrix + matrix.T) / 2  # Ensure symmetry

    # PyTorch
    pytorch_time, pytorch_current, pytorch_peak = pytorch_cpu_test(matrix)
    # TensorFlow
    tensorflow_time, tensorflow_current, tensorflow_peak = tensorflow_cpu_test(matrix)

    results["size"].append(size)
    results["pytorch"].append((pytorch_time, pytorch_current, pytorch_peak))
    results["tensorflow"].append((tensorflow_time, tensorflow_current, tensorflow_peak))

    print(f"Matrix size {size}x{size}:")
    print(f"  PyTorch: Time={pytorch_time:.4f}s")
    print(f"  TensorFlow: Time={tensorflow_time:.4f}s")


Testing matrix size: 10x10
Matrix size 10x10:
  PyTorch: Time=0.0043s
  TensorFlow: Time=0.0002s
Testing matrix size: 100x100
Matrix size 100x100:
  PyTorch: Time=0.0221s
  TensorFlow: Time=0.0011s
Testing matrix size: 500x500
Matrix size 500x500:
  PyTorch: Time=0.0400s
  TensorFlow: Time=0.0514s
Testing matrix size: 1000x1000
Matrix size 1000x1000:
  PyTorch: Time=0.2950s
  TensorFlow: Time=0.4868s
Testing matrix size: 2500x2500
Matrix size 2500x2500:
  PyTorch: Time=3.8300s
  TensorFlow: Time=10.5320s
Testing matrix size: 5000x5000
Matrix size 5000x5000:
  PyTorch: Time=36.1616s
  TensorFlow: Time=103.6179s


In [1]:
# Experiment on (Numpy), PyTorch, and TensorFlow on GPU runtime
import numpy as np
import torch
import tensorflow as tf
import time

# Function to measure memory and time for NumPy
def numpy_memory_test(matrix):
    start_time = time.time()
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    end_time = time.time()
    return end_time - start_time, None, None  # NumPy does not provide memory tracking

# Function to measure memory and time for PyTorch
def pytorch_memory_test(matrix):
    torch_matrix = torch.tensor(matrix, dtype=torch.float32, device="cuda")
    torch.cuda.reset_peak_memory_stats()
    start_time = time.time()
    eigenvalues, eigenvectors = torch.linalg.eigh(torch_matrix)
    end_time = time.time()
    current_memory = torch.cuda.memory_allocated()
    peak_memory = torch.cuda.max_memory_allocated()
    return end_time - start_time, current_memory, peak_memory

# Function to measure memory and time for TensorFlow
def tensorflow_memory_test(matrix):
    tf_matrix = tf.convert_to_tensor(matrix, dtype=tf.float32)
    tf.config.experimental.reset_memory_stats("GPU:0")
    start_time = time.time()
    eigenvalues, eigenvectors = tf.linalg.eigh(tf_matrix)
    end_time = time.time()
    memory_info = tf.config.experimental.get_memory_info("GPU:0")
    current_memory = memory_info["current"]
    peak_memory = memory_info["peak"]
    return end_time - start_time, current_memory, peak_memory

# Testing matrix sizes
matrix_sizes = [10, 100, 500, 1000, 2500, 5000]
results = {"size": [], "numpy": [], "pytorch": [], "tensorflow": []}

# Run experiments
for size in matrix_sizes:
    print(f"Testing matrix size: {size}x{size}")
    matrix = np.random.rand(size, size)
    matrix = (matrix + matrix.T) / 2  # Ensure symmetry

    # NumPy
    numpy_time, numpy_current, numpy_peak = numpy_memory_test(matrix)
    # PyTorch
    pytorch_time, pytorch_current, pytorch_peak = pytorch_memory_test(matrix)
    # TensorFlow
    tensorflow_time, tensorflow_current, tensorflow_peak = tensorflow_memory_test(matrix)

    results["size"].append(size)
    results["numpy"].append((numpy_time, numpy_current, numpy_peak))
    results["pytorch"].append((pytorch_time, pytorch_current, pytorch_peak))
    results["tensorflow"].append((tensorflow_time, tensorflow_current, tensorflow_peak))

    print(f"Matrix size {size}x{size}:")
    print(f"  NumPy: Time={numpy_time:.4f}s")
    print(f"  PyTorch: Time={pytorch_time:.4f}s, Current={pytorch_current/1e6:.2f}MB, Peak={pytorch_peak/1e6:.2f}MB")
    print(f"  TensorFlow: Time={tensorflow_time:.4f}s, Current={tensorflow_current/1e6:.2f}MB, Peak={tensorflow_peak/1e6:.2f}MB")


Testing matrix size: 10x10
Matrix size 10x10:
  NumPy: Time=0.0149s
  PyTorch: Time=0.5725s, Current=0.00MB, Peak=0.14MB
  TensorFlow: Time=0.2235s, Current=0.00MB, Peak=0.14MB
Testing matrix size: 100x100
Matrix size 100x100:
  NumPy: Time=0.0222s
  PyTorch: Time=0.0345s, Current=0.08MB, Peak=0.14MB
  TensorFlow: Time=0.0060s, Current=0.08MB, Peak=0.47MB
Testing matrix size: 500x500
Matrix size 500x500:
  NumPy: Time=0.0925s
  PyTorch: Time=0.0699s, Current=2.00MB, Peak=3.08MB
  TensorFlow: Time=0.1158s, Current=2.10MB, Peak=11.49MB
Testing matrix size: 1000x1000
Matrix size 1000x1000:
  NumPy: Time=0.4309s
  PyTorch: Time=0.1713s, Current=8.00MB, Peak=20.98MB
  TensorFlow: Time=0.0452s, Current=8.20MB, Peak=29.37MB
Testing matrix size: 2500x2500
Matrix size 2500x2500:
  NumPy: Time=3.7158s
  PyTorch: Time=0.2419s, Current=50.34MB, Peak=125.84MB
  TensorFlow: Time=0.1916s, Current=58.57MB, Peak=234.89MB
Testing matrix size: 5000x5000
Matrix size 5000x5000:
  NumPy: Time=41.3265s
  PyT